## Imports

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
from fredapi import Fred
from dotenv import load_dotenv
import os
import json
from functools import reduce
import numpy as np


load_dotenv()

FRED_API_KEY = os.getenv("FRED_API_KEY")

fred = Fred(api_key=FRED_API_KEY)

## Helpers

In [2]:
def series_to_json(series, df):
    """
    Convert a FRED series to a JSON string.
    """
    path = f"datasets/fred_{series}.json"
    # If file exists, load and merge
    if os.path.exists(path):
        # Load existing data
        with open(path, "r") as f:
            existing_data = json.load(f)
        df_existing = pd.DataFrame(existing_data)
        df_existing["Date"] = pd.to_datetime(df_existing["Date"])

        # Append only new rows
        df_combined = pd.concat([df_existing, df])
        df_combined = df_combined.drop_duplicates(subset=["Date"]).sort_values("Date")
    else:
        # No file exists, use fresh data
        df_combined = df

    # Save to JSON
    df_combined.to_json(path, orient="records", date_format="iso")

    print(f"Updated data saved to {path}")


def merge_on_year(dfs, how='inner'):
    """
    Merge a list of dataframes on the 'Year' column.

    Parameters:
    dfs (list): List of pandas DataFrames, each with a 'Year' column.
    how (str): Type of merge - 'inner', 'outer', 'left', 'right'. Default 'inner'.

    Returns:
    DataFrame: Merged dataframe.
    """
    # Safety check: make sure they all have 'Year' column
    for i, df in enumerate(dfs):
        if 'Year' not in df.columns:
            raise ValueError(f"DataFrame at index {i} is missing 'Year' column.")

    merged_df = reduce(lambda left, right: pd.merge(left, right, on='Year', how=how), dfs)
    return merged_df


def calc_mtg_pi_payment(principal, annual_rate, years=30):
    """
    Calculate monthly principal & interest payment for a mortgage.

    principal: Loan amount
    annual_rate: Annual interest rate as a percentage (e.g., 6.5 for 6.5%)
    years: Term in years (default 30)
    """
    monthly_rate = (annual_rate / 100) / 12
    n_payments = years * 12
    
    if monthly_rate == 0:
        return principal / n_payments  # handle zero interest edge case
    
    payment = principal * (monthly_rate * (1 + monthly_rate) ** n_payments) / \
              ((1 + monthly_rate) ** n_payments - 1)
    return payment


In [3]:
def scale_for_inflation(cpi_df: pd.DataFrame, from_year: int, to_year: int, amount: float):
    from_year_cpi = cpi_df.loc[cpi_df['Year'] == from_year, 'CPI'].values[0]
    to_year_cpi = cpi_df.loc[cpi_df['Year'] == to_year, 'CPI'].values[0]
    adjusted_value = (amount * (to_year_cpi / from_year_cpi))
    
    return round(adjusted_value, 2)


## Affordability

### Producer Price Index by Industry: Premiums for Property and Casualty Insurance: Premiums for Homeowner's Insurance (PCU9241269241262)

In [ ]:
cpi = fred.get_series('CPIAUCSL')
cpi_df = cpi.to_frame().reset_index()
cpi_df.columns = ['Date', 'CPI']
cpi_df['Date'] = pd.to_datetime(cpi_df['Date'])
cpi_df.set_index('Date', inplace=True)
cpi_df = cpi_df.resample('YE').max().round(2)
cpi_df.index = cpi_df.index.year
cpi_df.reset_index(inplace=True)
cpi_df.columns = ['Year', 'CPI']

cpi_df.tail()

In [6]:
hoi_ref_premium = 3303
hoi_ref_year = 2024

#CPI table - resampled to annual on mean
cpi = fred.get_series('CPIAUCSL')
cpi_df = cpi.to_frame().reset_index()
cpi_df.columns = ['Date', 'CPI']
cpi_df['Date'] = pd.to_datetime(cpi_df['Date'])
cpi_df.set_index('Date', inplace=True)
cpi_df = cpi_df.resample('YE').max()
cpi_df.index = cpi_df.index.year
cpi_df.reset_index(inplace=True)
cpi_df.columns = ['Year', 'CPI']

#HOI PPI table - resampled to annual on mean
hoi_series = fred.get_series('PCU9241269241262')
hoi_df = hoi_series.to_frame().reset_index()
hoi_df.columns = ['Date', 'HOI PPI']
hoi_df['Date'] = pd.to_datetime(hoi_df['Date'])
hoi_df.set_index('Date', inplace=True)
hoi_df = hoi_df.resample('YE').mean()
hoi_df.index = hoi_df.index.year
hoi_df.reset_index(inplace=True)
hoi_df.columns = ['Year', 'HOI PPI']

#Estimate HOI premiums based on reference year adjusted by PPI
hoi_ref_cpi = hoi_df.loc[hoi_df['Year'] == hoi_ref_year, 'HOI PPI'].values[0]
hoi_df['Est HOI Premium'] = (hoi_df['HOI PPI'] * (hoi_ref_premium / hoi_ref_cpi))

#Merge the datasets
merged_hoi_df = merge_on_year([hoi_df, cpi_df], how='outer')

# anchor year where both PPI + Premium exist
anchor_year = 1998
premium_anchor = merged_hoi_df.loc[merged_hoi_df["Year"] == anchor_year, "Est HOI Premium"].values[0]
cpi_anchor = merged_hoi_df.loc[merged_hoi_df["Year"] == anchor_year, "CPI"].values[0]

# fill missing premiums before PPI begins
mask = merged_hoi_df["Year"] < anchor_year
merged_hoi_df.loc[mask, "Est HOI Premium"] = (
    premium_anchor * (merged_hoi_df.loc[mask, "CPI"] / cpi_anchor)
)

merged_hoi_df.loc[mask, "HOI PPI"] = np.nan
merged_hoi_df['Scaled Premium'] = merged_hoi_df.apply(lambda row: scale_for_inflation(cpi_df, 2024, row['Year'], row['Est HOI Premium']), axis=1)

# merged_hoi_df.tail()
median_home_prices = fred.get_series('MSPUS')
df_home_median_prices = median_home_prices.to_frame().reset_index()
df_home_median_prices.columns = ['Date', 'Median Sales Price']
df_home_median_prices['Date'] = pd.to_datetime(df_home_median_prices['Date'])
df_home_median_prices.set_index('Date', inplace=True)

df_home_median_prices_annual = df_home_median_prices.resample('YE').mean()
df_home_median_prices_annual.index = df_home_median_prices_annual.index.year
df_home_median_prices_annual.reset_index(inplace=True)
df_home_median_prices_annual.columns = ['Year', 'Median Sales Price']
#df_home_median_prices_annual.tail()

median_family_income = fred.get_series('MEFAINUSA646N')
df_median_family_income =  median_family_income.to_frame().reset_index()
df_median_family_income.columns = ['Date', 'Median Family Income']
df_median_family_income['Date'] = pd.to_datetime(df_median_family_income['Date'])
df_median_family_income.set_index('Date', inplace=True)
df_median_family_income.index = df_median_family_income.index.year
df_median_family_income.reset_index(inplace=True)
df_median_family_income.columns = ['Year', 'Median Family Income']

mtg30 = fred.get_series('MORTGAGE30US')
df_mtg30 = mtg30.to_frame().reset_index()
df_mtg30.columns = ['Date', '30yr Mtg Rate']
df_mtg30['Date'] = pd.to_datetime(df_mtg30['Date'])
df_mtg30.set_index('Date', inplace=True)
df_mtg30 = df_mtg30.resample('YE').mean()
df_mtg30.index = df_mtg30.index.year
df_mtg30.reset_index(inplace=True)
df_mtg30.columns = ['Year', '30yr Mtg Rate']
df_mtg30['30yr Mtg Rate'] = round(df_mtg30['30yr Mtg Rate'], 3)

cdf = merge_on_year([merged_hoi_df, df_home_median_prices_annual, df_median_family_income, df_mtg30])
cdf['Avg Loan Amount'] = cdf['Median Sales Price'] * .8
cdf['Mtg PI Annual'] = cdf.apply(lambda row: calc_mtg_pi_payment(row['Avg Loan Amount'] * 12, row['30yr Mtg Rate']), axis=1).round(2)
cdf['Mtg PII Annual'] = cdf['Mtg PI Annual'] + cdf['Scaled Premium']
cdf['Mtg PII Monthly'] = round((cdf['Mtg PI Annual'] / 12) + (cdf['Scaled Premium'] / 12), 2)
cdf['Mtg Ratio'] = cdf['Mtg PII Annual'] / cdf['Median Family Income']
cdf.tail(50)

,Year,HOI PPI,Est HOI Premium,CPI,Scaled Premium,Median Sales Price,Median Family Income,30yr Mtg Rate,Avg Loan Amount,Mtg PI Annual,Mtg PII Annual,Mtg PII Monthly,Mtg Ratio
3,1974,NaN,419.859487,51.900,68.61,36050.0,12900.0,9.187,28840.0,2831.33,2899.94,241.66,0.224802
4,1975,NaN,449.791666,55.600,78.74,39275.0,13720.0,9.047,31420.0,3046.51,3125.25,260.44,0.227788
5,1976,NaN,472.443045,58.400,86.87,44225.0,14960.0,8.866,35380.0,3375.25,3462.12,288.51,0.231425
6,1977,NaN,503.993179,62.300,98.86,48900.0,16010.0,8.845,39120.0,3724.98,3823.84,318.65,0.238841
7,1978,NaN,549.295937,67.900,117.43,55850.0,17640.0,9.642,44680.0,4563.98,4681.41,390.12,0.265386
8,1979,NaN,622.103941,76.900,150.63,62750.0,19590.0,11.204,50200.0,5829.85,5980.48,498.37,0.305282
9,1980,NaN,698.956833,86.400,190.14,64750.0,21020.0,13.742,51800.0,7238.42,7428.56,619.05,0.353404
10,1981,NaN,761.248125,94.100,225.54,68950.0,22390.0,16.642,55160.0,9244.67,9470.21,789.18,0.422966
11,1982,NaN,793.607238,98.100,245.13,69225.0,23430.0,16.044,55380.0,8960.30,9205.43,767.12,0.392891
12,1983,NaN,820.303505,101.400,261.90,75375.0,24580.0,13.235,60300.0,8137.59,8399.49,699.96,0.341721


In [ ]:
value = scale_for_inflation(cpi_df, 1976, 2025, 86.88)
value

### Median Home Prices

In [ ]:
# Median Home Price
# Fred series MSPUS
median_home_prices = fred.get_series('MSPUS')
df_home_median_prices = median_home_prices.to_frame().reset_index()
df_home_median_prices.columns = ['Date', 'Median Sales Price']
df_home_median_prices['Date'] = pd.to_datetime(df_home_median_prices['Date'])
df_home_median_prices.set_index('Date', inplace=True)

df_home_median_prices_annual = df_home_median_prices.resample('YE').mean()
df_home_median_prices_annual.index = df_home_median_prices_annual.index.year
df_home_median_prices_annual.reset_index(inplace=True)
df_home_median_prices_annual.columns = ['Year', 'Median Sales Price']
df_home_median_prices_annual.tail()


### Median Family Income

In [ ]:
# Median Family Income in the United States (MEFAINUSA646N)
median_family_income = fred.get_series('MEFAINUSA646N')
df_median_family_income =  median_family_income.to_frame().reset_index()
df_median_family_income.columns = ['Date', 'Median Family Income']
df_median_family_income['Date'] = pd.to_datetime(df_median_family_income['Date'])
df_median_family_income.set_index('Date', inplace=True)
df_median_family_income.index = df_median_family_income.index.year
df_median_family_income.reset_index(inplace=True)
df_median_family_income.columns = ['Year', 'Median Family Income']

### Mortgage Rate - 30 year

In [ ]:
# 30 Year Mortgage Rate

mtg30 = fred.get_series('MORTGAGE30US')
df_mtg30 = mtg30.to_frame().reset_index()
df_mtg30.columns = ['Date', '30yr Mtg Rate']
df_mtg30['Date'] = pd.to_datetime(df_mtg30['Date'])
df_mtg30.set_index('Date', inplace=True)
df_mtg30 = df_mtg30.resample('YE').mean()
df_mtg30.index = df_mtg30.index.year
df_mtg30.reset_index(inplace=True)
df_mtg30.columns = ['Year', '30yr Mtg Rate']
df_mtg30['30yr Mtg Rate'] = round(df_mtg30['30yr Mtg Rate'], 3)

df_mtg30.tail()

In [ ]:
cdf = merge_on_year([merged_hoi_df, df_home_median_prices_annual, df_median_family_income, df_mtg30])
cdf['Avg Loan Amount'] = cdf['Median Sales Price'] * .8
cdf['Mtg PI Annual'] = cdf.apply(lambda row: calc_mtg_pi_payment(row['Avg Loan Amount'] * 12, row['30yr Mtg Rate']), axis=1).round(2)
cdf['Mtg PII Annual'] = cdf['Mtg PI Annual'] + cdf['Scaled Premium']
cdf['Mtg PII Monthly'] = round((cdf['Mtg PI Annual'] / 12) + (cdf['Scaled Premium'] / 12), 2)
cdf['Mtg Ratio'] = cdf['Mtg PII Annual'] / cdf['Median Family Income']
cdf.tail(50)

### US Population

In [ ]:
pop = fred.get_series('POPTHM')
pop_df = pop.to_frame().reset_index()
pop_df.columns = ['Date', 'US Population']
pop_df['Date'] = pd.to_datetime(pop_df['Date'])
pop_df.set_index('Date', inplace=True)

pop_df = pop_df.resample('YE').max().round(0)
pop_df.index = pop_df.index.year
pop_df.reset_index(inplace=True)
pop_df.columns = ['Year', 'US Population']
pop_df['US Population'] = pop_df['US Population'] * 1000

pop_df.tail()

### Median Personal Income

In [ ]:
# Median Personal Income In the United States (MEPAINUSA646N)
median_pers_income = fred.get_series('MEPAINUSA646N')
df_median_pers_income = median_pers_income.to_frame().reset_index()
df_median_pers_income.columns = ['Date', 'Median Personal Income']
df_median_pers_income['Date'] = pd.to_datetime(df_median_pers_income['Date'])
df_median_pers_income.set_index('Date', inplace=True)
df_median_pers_income.index = df_median_pers_income.index.year
df_median_pers_income.reset_index(inplace=True)
df_median_pers_income.columns = ['Year', 'Median Personal Income']

df_median_pers_income.tail()

In [ ]:
# Mortgage Payment Burden

# Calc monthly P&I using MORTGAGE30US and 80% of median home price
# Calc monthly T&I using average homeowner's insurance and property taxes
# Annualize the monthly payment and divide by median income

### CPI (CPIAUCSL)

In [ ]:
cpi = fred.get_series('CPIAUCSL')
cpi_df = cpi.to_frame().reset_index()
cpi_df.columns = ['Date', 'CPI']
cpi_df['Date'] = pd.to_datetime(cpi_df['Date'])
cpi_df.set_index('Date', inplace=True)
cpi_df = cpi_df.resample('YE').max()
cpi_df.index = cpi_df.index.year
cpi_df.reset_index(inplace=True)
cpi_df.columns = ['Year', 'CPI']

cpi_df.tail()

### Personal Consumption Expenditures (PCE)

In [ ]:
pce = fred.get_series('PCE')
pce_df = pce.to_frame().reset_index()
pce_df.columns = ['Date', 'PCE']
pce_df['Date'] = pd.to_datetime(pce_df['Date'])
pce_df.set_index('Date', inplace=True)
pce_df = round(pce_df.resample('YE').mean(), 2)
pce_df.index = pce_df.index.year
pce_df.reset_index(inplace=True)
pce_df.columns = ['Year', 'PCE']
pce_df['PCE'] = pce_df['PCE'] * 1000000000


pce_df.tail()

### Expenditures: Household Operations: All Consumer Units (CXUHHOPERLB0101M)

In [ ]:
hh_spend = fred.get_series('CXUHHOPERLB0101M')
hh_spend_df = hh_spend.to_frame().reset_index()
hh_spend_df.columns = ['Date', 'HH Spend Annual']
hh_spend_df['Date'] = pd.to_datetime(hh_spend_df['Date'])
hh_spend_df.set_index('Date', inplace=True)
hh_spend_df.index = hh_spend_df.index.year
hh_spend_df.reset_index(inplace=True)
hh_spend_df.columns = ['Year', 'HH Spend Annual']

hh_spend_df.tail()

### Expenditures: Vehicle Insurance: All Consumer Units (CXU500110LB0101M)

In [ ]:
vehicle_ins = fred.get_series('CXU500110LB0101M')
vehicle_ins_df = vehicle_ins.to_frame().reset_index()
vehicle_ins_df.columns = ['Date', 'Vehilcle Ins Annual']
vehicle_ins_df['Date'] = pd.to_datetime(vehicle_ins_df['Date'])
vehicle_ins_df.set_index('Date', inplace=True)
vehicle_ins_df.index = vehicle_ins_df.index.year
vehicle_ins_df.reset_index(inplace=True)
vehicle_ins_df.columns = ['Year', 'Vehicle Ins Annual']

vehicle_ins_df.tail()

### Total Households (TTLH)

In [ ]:
hh = fred.get_series("TTLHH")
hh_df = hh.to_frame().reset_index()
hh_df.columns = ['Date', 'US Households']
hh_df['Date'] = pd.to_datetime(hh_df['Date'])
hh_df.set_index('Date', inplace=True)
hh_df.index = hh_df.index.year
hh_df.reset_index(inplace=True)
hh_df.columns = ['Year', 'US Households']
hh_df['US Households'] = hh_df['US Households'] * 1000

hh_df.tail()

### PCE Services: Healthcare (DHLCRC1Q027SBEA)

In [ ]:
pce_healthcare = fred.get_series("DHLCRC1Q027SBEA")
pce_healthcare_df = pce_healthcare.to_frame().reset_index()
pce_healthcare_df.columns = ['Date', 'PCE Healthcare']
pce_healthcare_df['Date'] = pd.to_datetime(pce_healthcare_df['Date'])
pce_healthcare_df.set_index('Date', inplace=True)
pce_healthcare_df = round(pce_healthcare_df.resample('YE').mean(), 2)
pce_healthcare_df.index = pce_healthcare_df.index.year
pce_healthcare_df.reset_index(inplace=True)
pce_healthcare_df.columns = ['Year', 'PCE Healthcare']
pce_healthcare_df['PCE Healthcare'] = pce_healthcare_df['PCE Healthcare'] * 1000000000

pce_healthcare_df.tail()

### CPI Used Cars and Trucks (CUSR0000SETA02)

In [ ]:
auto_cpi = fred.get_series('CUSR0000SETA02')
auto_cpi_df = auto_cpi.to_frame().reset_index()
auto_cpi_df.columns = ['Date', 'Used Auto CPI']
auto_cpi_df['Date'] = pd.to_datetime(auto_cpi_df['Date'])
auto_cpi_df.set_index('Date', inplace=True)
auto_cpi_df = auto_cpi_df.resample('YE').mean()
auto_cpi_df.index = auto_cpi_df.index.year
auto_cpi_df.reset_index(inplace=True)
auto_cpi_df.columns = ['Year', 'Used Auto CPI']


In [ ]:
used_car_ref_year = 2024
used_car_ref_price = 28472

ref_cpi = auto_cpi_df.loc[auto_cpi_df['Year'] == used_car_ref_year, 'Used Auto CPI'].values[0]
auto_cpi_df['Est Avg Used Car Price'] = round((auto_cpi_df['Used Auto CPI'] * (used_car_ref_price / ref_cpi)),2)
auto_cpi_df.tail()


### CPI New Cars and Trucks (CUUR0000SETA01)

In [ ]:
new_auto_cpi = fred.get_series('CUSR0000SETA01')
new_auto_cpi_df = new_auto_cpi.to_frame().reset_index()
new_auto_cpi_df.columns = ['Date', 'New Auto CPI']
new_auto_cpi_df['Date'] = pd.to_datetime(new_auto_cpi_df['Date'])
new_auto_cpi_df.set_index('Date', inplace=True)
new_auto_cpi_df = new_auto_cpi_df.resample('YE').mean()
new_auto_cpi_df.index = new_auto_cpi_df.index.year
new_auto_cpi_df.reset_index(inplace=True)
new_auto_cpi_df.columns = ['Year', 'New Auto CPI']

In [ ]:
new_car_ref_year = 2024
new_car_ref_price = 48397

new_car_ref_cpi = new_auto_cpi_df.loc[new_auto_cpi_df['Year'] == new_car_ref_year, 'New Auto CPI'].values[0]
new_auto_cpi_df['Est Avg New Car Price'] = round((new_auto_cpi_df['New Auto CPI'] * (new_car_ref_price / ref_cpi)),2)
new_auto_cpi_df.tail()

### Real Disposable Personal Income (DSPI)

In [ ]:
rdpi = fred.get_series('DSPI')
rdpi_df = rdpi.to_frame().reset_index()
rdpi_df.columns = ['Date', 'RDPI']
rdpi_df['Date'] = pd.to_datetime(rdpi_df['Date'])
rdpi_df.set_index('Date', inplace=True)
rdpi_df = rdpi_df.resample('YE').max()
rdpi_df.index = rdpi_df.index.year
rdpi_df.reset_index(inplace=True)
rdpi_df.columns = ['Year', 'RDPI']
rdpi_df['RDPI'] = rdpi_df['RDPI'] * 1000000000

rdpi_df.tail()

### MERGED Affordability DF - ANNUAL 

In [ ]:
dfs = [df_home_median_prices_annual, df_median_family_income, rdpi_df, cpi_df, pce_df, hh_spend_df, vehicle_ins_df, df_mtg30, pop_df, hh_df, pce_healthcare_df]
df_merged = merge_on_year(dfs)
df_merged['Household Persons'] = round((df_merged['US Population'] / df_merged['US Households']), 2)
df_merged['RDPI Per Capita'] = round((df_merged['RDPI'] / df_merged['US Population']), 2)
#df_merged['PCE Indv Monthly'] = round(((df_merged['PCE'] / df_merged['US Population']) / 12),2)
#df_merged['PCE HH Monthly'] = round(((df_merged['PCE'] / df_merged['US Households']) / 12),2)
df_merged['PCE HH Healthcare Monthly'] = round((df_merged['PCE Healthcare'] / df_merged['US Households']) / 12,2)
#df_merged['PCE Indv Healthcare Monthly'] = round((df_merged['PCE Healthcare'] / df_merged['US Population']) / 12,2)
#df_merged['Home Price Family Income Multiplier'] = df_merged['Median Sales Price'] / df_merged['Median Family Income']
df_merged['Income Home Price Ratio'] = df_merged['Median Family Income'] / df_merged['Median Sales Price']
df_merged['Loan Amount'] = df_merged['Median Sales Price'] * .8
df_merged['Mtg P&I Annual'] = df_merged.apply(lambda row: calc_mtg_pi_payment(row['Loan Amount'] * 12, row['30yr Mtg Rate']), axis=1).round(2)
df_merged['Mtg to Fam Income Ratio'] = df_merged['Mtg P&I Annual'] / df_merged['Median Family Income']
#df_merged['Mtg to Pers Income Ratio'] = df_merged['Mtg P&I Annual'] / df_merged['Median Personal Income']
df_merged = df_merged.dropna(axis=0)
df_merged.tail(50)

In [ ]:
scale_for_inflation(cpi_df=cpi_df, from_year=1986, to_year=2025, amount=130000)

In [ ]:
# Expense to Income Ratios - goods variety

#vehicles, food, healthcare, etc..

# Disposable Income Share
# 100% - (sum of essential expenses)